<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/azure_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Azure AI model inference

This notebook explains how to use `llama-index-llm-azure-inference` package with models deployed with the Azure AI model inference API in Azure AI studio or Azure Machine Learning.

In [ ]:
%pip install llama-index-llms-azure-inference

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index

## Prerequisites

The Azure AI model inference is an API that allow developers to get access to a variety of models hosted on Azure AI using a consistent schema. You can use `llama-index-llms-azure-inference` integration package with models that support this API, including models deployed to Azure AI serverless API endpoints and a subset of models from Managed Inference. To read more about the API specification and the models that support it see [Azure AI model inference API](https://aka.ms/azureai/modelinference).

To run this tutorial you need:

1. Create an [Azure subscription](https://azure.microsoft.com).
2. Create an Azure AI hub resource as explained at [How to create and manage an Azure AI Studio hub](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-azure-ai-resource).
3. Deploy one model supporting the Azure AI model inference API. In this example we use a `Mistral-Large` deployment. You can follow the instructions at [Deploy models as serverless APIs](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-serverless).



## Environment Setup

Follow this steps to get the information you need from the model you want to use:

1. Go to the [Azure AI studio](https://ai.azure.com/) or [Azure Machine Learning studio](https://ml.azure.com), depending on the product you are using.
2. Go to deployments (endpoints in Azure Machine Learning) and select the model you have deployed as indicated in the prerequisites.
3. Copy the endpoint URL and the key.
    
> If your model was deployed with Microsoft Entra ID support, you don't need a key.

In this scenario, we have placed both the endpoint URL and key in the following environment variables:

In [ ]:
import os

os.environ["AZURE_INFERENCE_ENDPOINT"] = "<your-endpoint>"
os.environ["AZURE_INFERENCE_CREDENTIAL"] = "your-credential"

## Use the model

To use LLMs deployed in Azure AI studio or Azure Machine Learning you need the endpoint and credentials to connect to it. In Azure AI, endpoints can serve a single model so the parameter `model_name` is not always required.

In [ ]:
from llama_index.llms.azure_inference import AzureAICompletionsModel

In [ ]:
llm = AzureAICompletionsModel(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"],
    credential=os.environ["AZURE_INFERENCE_CREDENTIAL"],
)

Alternatively, if you endpoint support Microsof Entra ID, you can use the following code to create the client:

In [ ]:
from azure.identity import DefaultAzureCredential

llm = AzureAICompletionsModel(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

If you are planning to use asynchronous calling, it's a best practice to use the asynchronous version for the credentials:

In [ ]:
from azure.identity.aio import (
    DefaultAzureCredential as DefaultAzureCredentialAsync,
)

llm = AzureAICompletionsModel(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"],
    credential=DefaultAzureCredentialAsync(),
)

Use the `complete` endpoint for text completion

In [ ]:
response = llm.complete("The sky is a beautiful blue and")
print(response)

In [ ]:
response = llm.stream_complete("The sky is a beautiful blue and")
for r in response:
    print(r.delta, end="")

Use the `chat` endpoint for conversation

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with colorful personality."
    ),
    ChatMessage(role="user", content="Hello"),
]

response = llm.chat(messages)
print(response)

In [ ]:
response = llm.stream_chat(messages)
for r in response:
    print(r.delta, end="")

Rather than adding same parameters to each chat or completion call, you can set them at the client instance.

In [ ]:
llm = AzureAICompletionsModel(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"],
    credential=os.environ["AZURE_INFERENCE_CREDENTIAL"],
    temperature=0.0,
    # safe_prompt is a parameter supported only on Mistral family of models
    model_extras={"safe_prompt": True},
)

In [ ]:
response = llm.complete("The sky is a beautiful blue and")
print(response)